In [1]:
import pandas as pd

# İlk CSV dosyasını yükle
df_text = pd.read_csv('./preprocessed_data/lemmatized_and_misspelled_removed_SEFACED.csv')

# Ek özellikleri içeren ikinci CSV dosyasını yükle
df_features = pd.read_csv('./data_with_extra_features/SEFACED_all_features.csv')

# İlk DataFrame'den "Text" sütununu kaldır
df_features.drop(columns=['Text'], inplace=True)
df_features.drop(columns=['Class_Label'], inplace=True)

# Varsayılan olarak, her iki DataFrame'deki her satırın dizini aynı örneğe karşılık gelir,
# bu nedenle iki DataFrame'i dizinlerine göre birleştirebiliriz
df_merged = pd.concat([df_text, df_features], axis=1)

# Birleştirilmiş DataFrame'i yeni bir CSV dosyasına kaydet
df_merged.to_csv('merged_data.csv', index=False)



In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Input, Concatenate
from sklearn.metrics import classification_report
import numpy as np

# Birleştirilmiş CSV dosyasını yükleyin; önişlenmiş metin ve ek özellikleri içerir
df = pd.read_csv('merged_data.csv')

# Önişlenmiş metin, ek özellikler ve etiketleri çıkarın
X_text = df['lemmatized_tokens']
X_features = df[['html_tag_count', 'css_tag_count', 'spam_phrase_count', 'link_count', 'grammar_error_count', 'misspelled_word_count']]
y = df['Class_Label']

# Etiketleri sayısal değerlere dönüştürün
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Maksimum dizi uzunluğunu tanımlayın
max_len = 100  # Gerektiğinde ayarlayın

# Dizileri belirteçlere bölen ve dolduran
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_text)
X_text = tokenizer.texts_to_sequences(X_text)
X_text = pad_sequences(X_text, maxlen=max_len)

# Kelime dağarcığının boyutunu alın
vocab_size = len(tokenizer.word_index) + 1  # Dolgu belirteci için 1 ekleyin

# Veriyi eğitim ve test kümelerine ayırın
X_text_train, X_text_test, X_features_train, X_features_test, y_train, y_test = train_test_split(X_text, X_features, y, test_size=0.2, random_state=42)

# CNN model mimarisini tanımlayın
embedding_dim = 100  # Gerektiğinde ayarlayın
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

# Ek özellikler için giriş katmanını tanımlayın
feature_input = Input(shape=(X_features.shape[1],))

# CNN çıktısını ek özelliklerle birleştirin
concatenated = Concatenate()([model.output, feature_input])

# Daha fazla işlem yapmak için yoğun katmanlar ekleyin
concatenated = Dense(64, activation='relu')(concatenated)
output = Dense(1, activation='sigmoid')(concatenated)

# Modeli tanımlayın
model = Model(inputs=[model.input, feature_input], outputs=output)

# Modeli derleyin
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Modeli eğitin
model.fit([X_text_train, X_features_train], y_train, epochs=10, batch_size=32, validation_data=([X_text_test, X_features_test], y_test))

# Modeli değerlendirin
loss, accuracy = model.evaluate([X_text_test, X_features_test], y_test)
print(f'Test Kaybı: {loss}, Test Doğruluğu: {accuracy}')

# Sınıflandırma raporunu hesaplayın ve yazdırın
y_pred = model.predict([X_text_test, X_features_test])
y_pred_classes = (y_pred > 0.5).astype(int)
print("Classification Report:")
print(classification_report(y_test, y_pred_classes, target_names=label_encoder.classes_))


Epoch 1/10
200/200 [==============================] - 15s 55ms/step - loss: 0.2195 - accuracy: 0.9003 - val_loss: 0.0714 - val_accuracy: 0.9737
Epoch 2/10
200/200 [==============================] - 11s 55ms/step - loss: 0.0364 - accuracy: 0.9864 - val_loss: 0.0543 - val_accuracy: 0.9781
Epoch 3/10
200/200 [==============================] - 11s 54ms/step - loss: 0.0211 - accuracy: 0.9917 - val_loss: 0.0678 - val_accuracy: 0.9781
Epoch 4/10
200/200 [==============================] - 11s 53ms/step - loss: 0.0168 - accuracy: 0.9917 - val_loss: 0.0784 - val_accuracy: 0.9737
Epoch 5/10
200/200 [==============================] - 11s 53ms/step - loss: 0.0166 - accuracy: 0.9922 - val_loss: 0.0786 - val_accuracy: 0.9725
Epoch 6/10
200/200 [==============================] - 11s 54ms/step - loss: 0.0159 - accuracy: 0.9930 - val_loss: 0.0792 - val_accuracy: 0.9725
Epoch 7/10
200/200 [==============================] - 11s 55ms/step - loss: 0.0149 - accuracy: 0.9930 - val_loss: 0.0891 - val_accuracy: